<a href="https://colab.research.google.com/github/AntoinettedeLima/A-mouse-using-your-eyes./blob/main/FYP_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MOUNTING THE DRIVE TO LOAD THE DATA SET**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# To read and write geospatial raster data, specifically designed to handle GeoTIFF. (to open .tif files)
!pip install rasterio

#  to apply operations like erosion in the Dark Channel Prior function.
!pip install opencv-python rasterio tensorflow


In [ ]:
# Import necessary libraries
import os
from glob import glob
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Set the directory where fog images are stored
fog_data_dir = '/content/drive/MyDrive/Duweeja de Lima - FYP Folder - 20210522/Data sets/FOGG DATA'  # Update this path
save_dir = '/content/drive/MyDrive/Duweeja de Lima - FYP Folder - 20210522/Data sets/SORTED FOG DATA'  # Update this path
os.makedirs(save_dir, exist_ok=True)


In [ ]:
# Get a list of all .tif files in the directory and map indexes to file paths
fog_images = glob(os.path.join(fog_data_dir, '*.tif'))
indexed_images = {}

In [ ]:
# Extract index from filename for sorting
for img in fog_images:
    match = re.search(r'(\d+)', os.path.basename(img))
    if match:
        index = int(match.group(1))
        indexed_images[index] = img

# Sort images by index
sorted_images = [indexed_images[index] for index in sorted(indexed_images.keys())]

**DEFINNING NORMALIZATION AND DARK CHANNEL PRIOR FUNCTIONS**

In [ ]:
# Function to normalize images
def normalize_image(img):
    return (img - img.min()) / (img.max() - img.min())

# DCP function for defogging
def dark_channel_prior(image, patch_size=15):
    min_channel = np.min(image, axis=2)
    dark_channel = cv2.erode(min_channel, np.ones((patch_size, patch_size)))
    flat_dark = dark_channel.ravel()
    sorted_indices = np.argsort(flat_dark)[::-1]
    n_pixels = int(0.001 * len(flat_dark))
    atmospheric_light = np.mean(image.reshape(-1, 3)[sorted_indices[:n_pixels]], axis=0)
    transmission = 1 - 0.95 * (dark_channel / atmospheric_light.max())
    transmission = np.clip(transmission, 0.1, 0.9)
    result = np.empty_like(image)
    for i in range(3):
        result[:, :, i] = (image[:, :, i] - atmospheric_light[i]) / transmission + atmospheric_light[i]
    return np.clip(result, 0, 255).astype(np.uint8)


**PROCESSING IMAGES IN BATCHES**

In [ ]:
# Function to process images in batches to avoid memory overload
def process_images_in_batches(image_paths, batch_size=50):
    processed_images = []
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        batch_processed = []
        for file_path in batch:
            with rasterio.open(file_path) as img:
                image_data = img.read([1, 2, 3])  # Assume RGB bands
                image_data = np.moveaxis(image_data, 0, -1)  # Rearrange for DCP (H, W, C)
                dcp_image = dark_channel_prior(image_data)
                normalized_image = normalize_image(dcp_image)
                batch_processed.append(normalized_image)
        processed_images.extend(batch_processed)
    return processed_images

# Apply the processing function in batches
preprocessed_images = process_images_in_batches(sorted_images, batch_size=50)


-------------------------------------------------

In [ ]:
# Get a list of all .tif files in the directory
#fog_images = glob(os.path.join(fog_data_dir, '*.tif'))

# Display the number of images loaded
#print(f"Number of fog images: {len(fog_images)}")


In [ ]:
# Print the indexes of all images
print("Indexes of fog images:")
for index, image_path in enumerate(fog_images):
    print(f"Index {index}: {os.path.basename(image_path)}")


In [ ]:
# Extract indexes from filenames
indexes = []
for img in fog_images:
    # Extract numbers from filenames, assuming format 'image_<index>.tif'
    match = re.search(r'(\d+)', os.path.basename(img))
    if match:
        indexes.append(int(match.group(1)))

# Sort indexes to find missing numbers
indexes.sort()
missing_indexes = []

# Check for missing indexes in the sorted list
for i in range(indexes[0], indexes[-1] + 1):
    if i not in indexes:
        missing_indexes.append(i)

# Output results
if missing_indexes:
    print(f"Missing indexes: {missing_indexes}")
else:
    print("All indexes are present.")


In [ ]:
# Create a dictionary to map indexes to file paths
indexed_images = {}

for img in fog_images:
    # Extract numbers from filenames, assuming format 'image_<index>.tif'
    match = re.search(r'(\d+)', os.path.basename(img))
    if match:
        index = int(match.group(1))
        indexed_images[index] = img

# Sort images by index
sorted_images = [indexed_images[index] for index in sorted(indexed_images.keys())]

# Print the sorted list of file paths
print("Sorted list of images by index:")
for img_path in sorted_images:
    print(img_path)


In [ ]:
# Function to display a sample image
def display_image(file_path):
    with rasterio.open(file_path) as img:
        plt.imshow(img.read(1), cmap='gray')
        plt.title(f"Sample Image: {os.path.basename(file_path)}")
        plt.axis('off')
        plt.show()

# Display a sample image from the sorted list to inspect
display_image(sorted_images[0])  # Show the first image in the sorted list

**DATA NORMALIZATION**

In [ ]:
# Normalization function
# adjusting the pixel values of an image so that they lie within a specific range, typically between 0 and 1
def normalize_image(img):
    return (img - img.min()) / (img.max() - img.min())


**DARC CHANNEL PRIOR FOR DEFOGGING**

In [ ]:
# DCP function for defogging
def dark_channel_prior(image, patch_size=15):
    min_channel = np.min(image, axis=2)
    dark_channel = cv2.erode(min_channel, np.ones((patch_size, patch_size)))
    flat_dark = dark_channel.ravel()
    sorted_indices = np.argsort(flat_dark)[::-1]
    n_pixels = int(0.001 * len(flat_dark))
    atmospheric_light = np.mean(image.reshape(-1, 3)[sorted_indices[:n_pixels]], axis=0)
    transmission = 1 - 0.95 * (dark_channel / atmospheric_light.max())
    transmission = np.clip(transmission, 0.1, 0.9)
    result = np.empty_like(image)
    for i in range(3):
        result[:, :, i] = (image[:, :, i] - atmospheric_light[i]) / transmission + atmospheric_light[i]
    return np.clip(result, 0, 255).astype(np.uint8)

**RESIZING IMAGES**

In [ ]:
# Resize function
def resize_images(images, target_size=(256, 256)):
    resized_images = []
    for img in images:
        resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
        resized_images.append(resized_img)
    return resized_images

**APPLYING DCP, NORMALIZATION AND RESIZING TO ALL SORTED IMAGES**

In [ ]:
# Apply DCP, normalization, and resizing to all sorted images
preprocessed_images = []
for file_path in sorted_images:
    with rasterio.open(file_path) as img:
        image_data = img.read([1, 2, 3])  # Assume RGB bands
        image_data = np.moveaxis(image_data, 0, -1)  # Rearrange for DCP (H, W, C)
        dcp_image = dark_channel_prior(image_data)
        normalized_image = normalize_image(dcp_image)
        preprocessed_images.append(normalized_image)

# Resize all DCP-processed images
resized_fog_images = resize_images(preprocessed_images, target_size=(256, 256))
